<a href="https://colab.research.google.com/github/masayasato0407/sksurv_RSF2/blob/main/Scikit_survival_RSF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#以下URLをclone
!git clone https://github.com/masayasato0407/sksurv_RSF2

In [ ]:
cd sksurv_RSF2

In [ ]:
#requirements.txtの読み込み
!pip install -r requirements.txt

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.linear_model.coxph import BreslowEstimator
import sksurv
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest

In [ ]:
#説明変数(trainx)ファイルの読み込み
from google.colab import files
uploaded = files.upload()

In [ ]:
X_train0=pd.read_csv ("trainx.csv",encoding="utf-8")
print (X_train0)

In [ ]:
#ndarrrayへ変換
X_train = X_train0.to_numpy()
X_train

In [ ]:
#説明変数(testx)ファイルの読み込み
from google.colab import files
uploaded = files.upload()

In [ ]:
X_test0=pd.read_csv ("testx.csv",encoding="utf-8")
print (X_test0)

In [ ]:
#ndarrrayへ変換
X_test = X_test0.to_numpy()
X_test

In [ ]:
#timeとeventのdata, trainy読み込み
from google.colab import files
uploaded = files.upload()

In [ ]:
y_train0=pd.read_csv ("trainy.csv",encoding="utf-8")
print (y_train0)

In [ ]:
#ここから、SKsurvで解析可能な形にy0をyのadarrayとして出力する
#subsetはyファイルの変数
subset1 = y_train0[['event', 'time']]

In [ ]:
#tuppleのリストの作成
tuples1 = [tuple(x) for x in subset1.to_numpy()]

In [ ]:
#tupleリスト確認
tuples1

In [ ]:
#dtypeを指定
dt=np.dtype([('event', '?'), ('time', '<f8')])

In [ ]:
#作成したndarrayをyとする
y_train=np.array(tuples1, dtype=dt)

In [ ]:
#timeとeventのdata, testy読み込み
from google.colab import files
uploaded = files.upload()

In [ ]:
y_test0=pd.read_csv ("testy.csv",encoding="utf-8")
print (y_test0)

In [ ]:
#ここから、SKsurvで解析可能な形にy0をyのadarrayとして出力する
#subsetはyファイルの変数
subset2 = y_test0[['event', 'time']]

In [ ]:
tuples2 = [tuple(x) for x in subset2.to_numpy()]

In [ ]:
#作成したndarrayをyとする
y_test=np.array(tuples2, dtype=dt)

In [ ]:
#RFモデルの作成 (以下のパラメーターを指定、時間がかかるならn_estimatorsを10などに設定)
rsf = RandomSurvivalForest(n_estimators=100,
                           min_samples_split=10,
                           min_samples_leaf=20,
                           max_depth=20,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=12)
rsf.fit(X_train, y_train)

In [ ]:
#test (外部バリデーション)でのCindex確認
rsf.score(X_test, y_test)

In [ ]:
#train (内部バリデーション)でのCindex確認
rsf.score(X_train, y_train)

#Permutation importanceを用いた、変数重要度の計算

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
#trainXのpd.dataframeから変数の定義づけ (右端の変数を一旦落としまた入れる)
X_no_grade = X_train0.drop("GGT", axis=1)
feature_names = X_no_grade.columns.tolist() + ["GGT"]

In [ ]:
#feature_namesの中身確認
feature_names

In [ ]:
#permutation importanceの計算
result = permutation_importance(rsf, X_test, y_test, n_repeats=15, random_state=1)

In [ ]:
#結果をPandasのデータフレームに入れて、表示します
df = pd.DataFrame([feature_names,result.importances_mean,result.importances_std],index=['Featue','mean','std']).T
df_s = df.sort_values('mean',ascending=False)
print(df_s)

#ここから、実際の患者データにモデルを適用

In [ ]:
#cases (実際の患者データ)ファイルの読み込み
from google.colab import files
uploaded = files.upload()

In [ ]:
X_cases=pd.read_csv ("cases.csv",encoding="utf-8")
print(X_cases)

In [ ]:
#個別のカプランマイヤー
surv = rsf.predict_survival_function(X_cases, return_array=True)

for i, s in enumerate(surv):
    plt.step(rsf.unique_times_, s, where="post", label=str(i))
plt.ylabel("1-event")
plt.xlabel("years")
plt.legend()
plt.grid(True)

In [ ]:
#累積ハザードでの表示
surv = rsf.predict_cumulative_hazard_function(X_cases, return_array=True)

for i, s in enumerate(surv):
    plt.step(rsf.unique_times_, s, where="post", label=str(i))
plt.ylabel("Cumulative hazard")
plt.xlabel("years")
plt.legend()
plt.grid(True)